<a href="https://colab.research.google.com/github/ludwigwittgenstein2/Research/blob/master/Novel_Address_Using_LLM_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Title: Novel Address Generation using LLM and LSTM
### by Rick Rejeleene

Generative Models, mostly we are using to generate any artifact [text, audio, video]

## First we approach using GPT-2 Model - Why?

a. GPT-2 is popular model of recent 

b. We try to use our dataset on GPT-2

c. We try to get results



###Goal of the Work: 

a. Apply Language Model 

b. Apply LSTM or variation of LSTM

c. Use Census dataset [Name-Address]

d. Compare results 

e. Write Paper/Results


In [ ]:
#install dependencies 


In [ ]:
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

In [ ]:
#Check GPU 

In [ ]:
!nvidia-smi

Tue Nov 22 17:27:59 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 602Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 2.72Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 473Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:19, 26.0Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 777Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 3.11Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 3.63Mit/s]


In [ ]:
gpt2.mount_gdrive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#upload our own dataset 

In [ ]:
file_name = 'CleanedDatasetEntire1.txt'

In [ ]:
gpt2.copy_file_from_gdrive(file_name)

In [ ]:
#Train Model

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=1000,
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              sample_every=200,
              save_every=500
              )

Loading checkpoint models/124M/model.ckpt
Loading dataset...


100%|██████████| 1/1 [00:00<00:00, 1167.35it/s]

dataset has 17233 tokens
Training...


[10 | 26.24] loss=2.29 avg=2.29
[20 | 46.97] loss=1.94 avg=2.12
[30 | 68.01] loss=1.57 avg=1.93
[40 | 89.33] loss=1.44 avg=1.81
[50 | 110.97] loss=1.23 avg=1.69
[60 | 132.89] loss=1.14 avg=1.60
[70 | 155.11] loss=0.92 avg=1.50
[80 | 177.43] loss=0.75 avg=1.40
[90 | 199.60] loss=0.56 avg=1.30
[100 | 221.74] loss=0.39 avg=1.21
[110 | 244.05] loss=0.29 avg=1.12
[120 | 266.28] loss=0.37 avg=1.05
[130 | 288.50] loss=0.16 avg=0.98
[140 | 310.84] loss=0.14 avg=0.92
[150 | 333.06] loss=0.09 avg=0.86
[160 | 355.25] loss=0.07 avg=0.81
[170 | 377.45] loss=0.08 avg=0.76
[180 | 399.67] loss=0.05 avg=0.72
[190 | 421.85] loss=0.07 avg=0.68
[200 | 444.06] loss=0.06 avg=0.65
======== SAMPLE 1 ========
 9, 6963 N EL SAIPA DR, POMONA ,CA 91768"
54	"Ms. Jordan Adams , Sr, QC, "	"3824 GRANGER PL, SARASOTA ,FL 33703"
55	"Ms. Samuel Adams , Sr, QA, "	"2508 PARADISE CIR, KISSIMMEE ,FL 34741"
56	"Ms. Sidney Adams , Sr, IQCP, "	"340 W 70TH ST, HIALEAH ,FL 33012"
57	"Ms. Braxton Julian , Sr, CLIA, "	"929 TRAMELL

Instructions for updating:
Use standard file APIs to delete files with this prefix.


In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

In [ ]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

In [ ]:
import tensorflow as tf 
tf.compat.v1.reset_default_graph()

In [ ]:

sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-1000


In [ ]:
#Generate our data from Model

In [ ]:
"""

Consider parameters

length: Number of tokens to generate (default 1023, the maximum)
temperature: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
top_k: Limits the generated guesses to the top k guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set top_k=40)
top_p: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with top_p=0.9)
truncate: Truncates the input text until a given sequence, excluding that sequence (e.g. if truncate='<|endoftext|>', 
the returned text will include everything before the first <|endoftext|>). 
It may be useful to combine this with a smaller length if the input texts are short.
include_prefix: If using truncate and include_prefix=False, the specified prefix will not be included in the returned text.

"""

In [ ]:
gpt2.generate(sess, run_name='run1')

98ACCESS RD, APT 27, VACAVILLE ,CA 95687"
99	"Dean. Tara Borders , Sr, CLIA,"	"2120 W RANDALL W, GAINESVILLE ,FL 32641"
100	"Dean. Justin Bolton , Sr, QC,"	"11350 BRIAR OAK DR, ROYAL PALM BEACH ,FL 33411"
101	"Dr. Wade Bolton , Jr, QA,"	"14234 COHASSET ST, VAN NUYS ,CA 91405"
102	"Mr. Dylan Bolton , Jr, CLIA,"	"14234 COHASSET ST, VAN NUYS ,CA 91405"
103	"Dean. Mysterio Abolgo , Sr, QC,"	"368 REDWING CIR, CORONA ,CA 92880"
104	"Dr. Michael Abbatie , Jr, QA,"	"4552 ABRUZZI CIR, STOCKTON ,CA 95206"
105	"Col. Abigail Amstutz , Jr, CLIA,"	"STERWING CIR, MOUNTAIN VIEW ,CA 94040"
106	"MS. Erin Amstutz , Jr, QC,"	"7547 VAN NUYS ,CA 91405"
107	"Dean .iti . Arnaud Amstutz , Jr, QA,"	"17419 forward Arie Bivings RD, No 36, Auvergne ,DE 27 toers .eri .eri . AIG ,2729"
108	"Dean. Niki Adams , PhD, QC,"	"3874 STEPHENS ST, POMONA ,CA 91768"
109	"Dr. Kayla Adams , PhD, QA,"	"4552 ABRUZZI CIR, STOCKTON ,CA 95206"
110	"Col. John Adams , Jr, QC,"	"14234 COHASSET ST, VAN NUYS ,CA 91405"
111	"MS. Annie Adam

In [ ]:
gpt2.generate(sess,
              length=250,
              temperature=0.7,
              prefix="Taj Mahal",
              nsamples=5,
              batch_size=5
              )

Taj Mahal , Sr, CLIA,"	"950 N DUESENBERG DR, APT 11203, ONTARIO ,CA 91764"
154	"Col. Jessica Addaz , PhD, QC,"	"6161 NW 44TH TER, COCONUT CREEK ,FL 33073"
155	"MS. Reid Amber Barnard , PhD, QA,"	"36491 YAMAS DR, APT 3205, WILDOMAR ,CA 92595"
156	"Dean . Adam Barnard , Sr, QC,"	"17460 CAMINITO BAYA, SAN DIEGO ,CA 92127"
157	"Dean. Estela Barnard , Sr, QA,"	"24850 HANCOCK AVE, APT B207, MURRIETA ,CA 92562"
158	"Dean. Isabel Barnard , Sr, IQCP,"	"3802 SUDBURY DR, PASADENA ,TX 77504"
159	"Dr. Aaron Barnard , Sr, CLIA,"	"4870 HIDDEN HILL DR, FORT WORTH ,TX 76
Taj Mahal , Sr, CLIA,"	"9537 MISTY RIVER WAY, ELK GROVE ,CA 95624"
53	"Dean. Taylor Mead , PhD, QC,"	"216 1/2 N BREED ST, LOS ANGELES ,CA 90033"
54	"Dr. Evanoj Hokage , PhD, QA,"	"408 GRISWOLD ST, # 3, GLENDALE ,CA 91205"
55	"Dean. Trenton Haugh , PhD, QC,"	"430 N, APT J233, GARDEN GROVE ,CA 92880"
56	"Dr. Sienna Haugh , PhD, QA,"	"41511 ABRUZZI CIR, STOCKTON ,CA 95206"
57	"Col. Luke Haggis , PhD, IQCP,"	"1514 OLD LA BLANCA RD, DONNA ,

#Text Generation using LSTM

In [ ]:
f = open('/content/word.txt', 'r')
content = f.read()


In [ ]:

print('Corpus length:', len(text))

In [ ]:
import keras
import numpy as np

path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

In [ ]:
# Length of extracted character sequences
maxlen = 60

# We sample a new sequence every `step` characters
step = 3

# This holds our extracted sequences
sentences = []

# This holds the targets (the follow-up characters)
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

# List of unique characters in the corpus
chars = sorted(list(set(text)))
print(chars)

print('Unique characters:', len(chars))

# Dictionary mapping unique characters to their index in `chars`
char_indices = dict((char, chars.index(char)) for char in chars)

# Next, one-hot encode the characters into binary arrays.
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [ ]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [ ]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [ ]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
import random
import sys

for epoch in range(1, 60):
    print('epoch', epoch)
    
    # Fit the model for 1 epoch on the available training data
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    # Select a text seed at random
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        # We generate 400 characters
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()